In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [12]:
url_tab = 'https://www.resetera.com/threads/media-create-sales-week-53-2020-dec-28-jan-03.356956/'
url_tab = 'https://www.resetera.com/threads/media-create-sales-week-46-2018-nov-12-nov-18.82357/'

In [13]:
ustitles = requests.get(url_tab)
soup = BeautifulSoup(ustitles.text)
aa=soup.findAll("div", {"class": "bbWrapper"})
print(aa[0].text)

 
Media Create Sales: Week 46, 2018 (Nov 12 - Nov 18) 

01./00.  [NSW] Pokemon: Let's Go, Pikachu! / Let's Go, Eevee! # <RPG> (Pokemon Co.) {2018.11.16} (¥5.980) - 661.240 / NEW 
02./00.  [PS4] Fallout 76 # <RPG> (Bethesda Softworks) {2018.11.15} (¥7.980) - 73.489 / NEW 
03./03. [NSW] Super Mario Party <ETC> (Nintendo) {2018.10.05} (¥5.980) - 20.942 / 345.627 (-10%)
04./02. [PS4] Call of Duty: Black Ops IIII <ACT> (Sony Interactive Entertainment) {2018.10.12} (¥7.900) - 17.159 / 460.890 (-33%)
05./00.  [PS4] Hitman 2 <ADV> (Warner Entertainment Japan) {2018.11.15} (¥7.800) - 10.162 / NEW 
06./01. [3DS] Luigi's Mansion <ACT> (Nintendo) {2018.11.08} (¥4.980) - 9.320 / 36.689 (-66%)
07./05. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980) - 9.319 / 1.836.063 (+10%)
08./04. [PS4] Red Dead Redemption II # <ACT> (Take-Two Interactive Japan) {2018.10.26} (¥8.800) - 8.120 / 192.181 (-52%)
09./07. [NSW] Splatoon 2 # <ACT> (Nintendo) {2017.07.21} (¥5.980) - 7.992 / 2.698.886 (+18

In [14]:
tab_top=aa[0].text.split("Famitsu Sales: Week")[1]
#tab_top=tab_top.replace("[","###").replace("]","###").replace("<","###").replace("./","###")
#tab_top=tab_top.replace(">","/").replace("(","###").replace(")","###").replace("{","###").replace("}","###")
tab_top=tab_top.replace(" # ","").split("\n")
#re.split('; |, |\*|\n',a)
tab_top[2:32]

["01./00.  [NSW] Pokemon: Let's Go, Pikachu! / Let's Go, Eevee!<RPG> (Pokemon Co.) {2018.11.16} (¥5.980) - 664.198 / NEW <40-60%> ",
 '02./00.  [PS4] Fallout 76<RPG> (Bethesda Softworks) {2018.11.15} (¥7.980) - 60.222 / NEW <60-80%> ',
 '03./02. [NSW] Super Mario Party <ETC> (Nintendo) {2018.10.05} (¥5.980) - 17.486 / 326.510 <80-100%> (-15%)',
 '04./03. [PS4] Call of Duty: Black Ops IIII <ACT> (Sony Interactive Entertainment) {2018.10.12} (¥7.900) - 13.868 / 442.995 <80-100%> (-32%)',
 "05./01. [3DS] Luigi's Mansion <ACT> (Nintendo) {2018.11.08} (¥4.980) - 10.716 / 40.121 <40-60%> (-64%)",
 '06./00.  [PS4] Hitman 2 <ADV> (Warner Entertainment Japan) {2018.11.15} (¥7.800) - 10.688 / NEW <20-40%> ',
 '07./05. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980) - 9.948 / 1.849.467 <80-100%> (+14%)',
 '08./04. [PS4] Red Dead Redemption II<ACT> (Take-Two Interactive Japan) {2018.10.26} (¥8.800) - 7.642 / 189.465 <80-100%> (-48%)',
 '09./06. [NSW] Minecraft <ADV> (Microsoft Gam

In [15]:
clean_tab=tab_top[2:32]
clean_tab

["01./00.  [NSW] Pokemon: Let's Go, Pikachu! / Let's Go, Eevee!<RPG> (Pokemon Co.) {2018.11.16} (¥5.980) - 664.198 / NEW <40-60%> ",
 '02./00.  [PS4] Fallout 76<RPG> (Bethesda Softworks) {2018.11.15} (¥7.980) - 60.222 / NEW <60-80%> ',
 '03./02. [NSW] Super Mario Party <ETC> (Nintendo) {2018.10.05} (¥5.980) - 17.486 / 326.510 <80-100%> (-15%)',
 '04./03. [PS4] Call of Duty: Black Ops IIII <ACT> (Sony Interactive Entertainment) {2018.10.12} (¥7.900) - 13.868 / 442.995 <80-100%> (-32%)',
 "05./01. [3DS] Luigi's Mansion <ACT> (Nintendo) {2018.11.08} (¥4.980) - 10.716 / 40.121 <40-60%> (-64%)",
 '06./00.  [PS4] Hitman 2 <ADV> (Warner Entertainment Japan) {2018.11.15} (¥7.800) - 10.688 / NEW <20-40%> ',
 '07./05. [NSW] Mario Kart 8 Deluxe <RCE> (Nintendo) {2017.04.28} (¥5.980) - 9.948 / 1.849.467 <80-100%> (+14%)',
 '08./04. [PS4] Red Dead Redemption II<ACT> (Take-Two Interactive Japan) {2018.10.26} (¥8.800) - 7.642 / 189.465 <80-100%> (-48%)',
 '09./06. [NSW] Minecraft <ADV> (Microsoft Gam

In [16]:
clean_tab=tab_top[2:32]
for i in range(len(clean_tab)):
    temp = clean_tab[0].split("-",1)
    
#>>> '-'.join(sentence)
#'this-is-a-sentence'

In [17]:
def flatten_list(x):
    flat_list = []
    for el in x:
        if type(el) != list:
            flat_list.append(el)
        else:
            for el2 in el:
                flat_list.append(el2)
    return flat_list

In [20]:
temp1=clean_tab[0]
print(temp1)

temp2=clean_tab[2]
print(temp2)

temp=temp2

01./00.  [NSW] Pokemon: Let's Go, Pikachu! / Let's Go, Eevee!<RPG> (Pokemon Co.) {2018.11.16} (¥5.980) - 664.198 / NEW <40-60%> 
03./02. [NSW] Super Mario Party <ETC> (Nintendo) {2018.10.05} (¥5.980) - 17.486 / 326.510 <80-100%> (-15%)


In [130]:


temp=temp1

def mine_sw_info(temp):
    row = []
    # Postition and previous week position
    row.append( int(re.findall(r"[0-9]{2}",temp)[0]) )
    row.append( int(re.findall(r"[0-9]{2}",temp)[1]) )
    # Find all things between [ ] and obtain the first one to get the system
    row.append(re.findall(r"\[(.*?)\]",temp)[0])
    # Find all things between < > and obtain the penultimate one to get the genre
    row.append(re.findall(r"\<(.*?)\>",temp)[-2])
    # Find all things between { } and obtain the last one to get the day of release
    row.append(re.findall(r"\{(.*?)\}",temp)[-1])



    if re.findall(r"\((.*?)\)",temp)[-1].find("%") >= 0: # Not a new release
        # price
        row.append(  int(re.findall(r"\((.*?)\)",temp)[-2].replace(".","").replace("¥","")) )
        # Units this week
        row.append(  int(re.findall(r"\-(.*?)\/",temp)[-1].replace(".","")) )
        # Total units
        row.append(  int(re.findall(r"\/(.*?)\<",temp)[-1].replace(".","")) )
        # Percentage of sales
        row.append(re.findall(r"\<(.*?)\>",temp)[-1])
        # Change with respect to the last week in percentage
        row.append(  int(re.findall(r"\((.*?)\)",temp)[-1].replace("%","")) )
        # Add a boolean to indicate this is not a new release
        row.append(  False )
    else:
        # price
        row.append(  int(re.findall(r"\((.*?)\)",temp)[-1].replace(".","").replace("¥","")) )
        # Units this week
        row.append(  int(re.findall(r"\-(.*?)\/",temp)[-1].replace(".","")) )
        # Total Units = Units this week
        row.append(  int(re.findall(r"\-(.*?)\/",temp)[-1].replace(".","")) )
        # Percentage of sales
        row.append(re.findall(r"\<(.*?)\>",temp)[-1])
        # Change with respect to the last week in percentage, so put 0 to indicate new
        row.append( 0)
        # Add a boolean to indicate this is not a new release
        row.append(  True )
    
    # Lastly, the name of the game
    name=temp.split(f"[{row[2]}]")[-1].split(f"<{row[3]}>")[0]
    row.append(name.lstrip().rstrip())
    return row

def create_top30_list():
    # need to create the header and append the rows in one page
    return "work_in_progress"


print(mine_sw_info(temp1))
print(mine_sw_info(temp2))

[1, 0, 'NSW', 'RPG', '2018.11.16', 5980, 664198, 664198, '40-60%', 0, True, "Pokemon: Let's Go, Pikachu! / Let's Go, Eevee!"]
[3, 2, 'NSW', 'ETC', '2018.10.05', 5980, 17486, 326510, '80-100%', -15, False, 'Super Mario Party']


In [ ]:
# https://www.neogaf.com/threads/media-create-sales-archive-thread-2.416253/#post-147729422

In [ ]:
url="https://www.resetera.com/threads/media-create-sales-archive-thread.4115/"
ustitles = requests.get(url)
soup = BeautifulSoup(ustitles.text)
links=soup.findAll("a", {"class": "link link--internal"})  #class="link link--internal" 